# Homework 3. Pandas

## Important notes

1. *When you open this file on GitHub, copy the address to this file from the address bar of your browser. Now you can go to [Google Colab](https://colab.research.google.com/), click `File -> Open notebook -> GitHub`, paste the copied URL and click the search button (the one with the magnifying glass to the right of the search input box). Your personal copy of this notebook will now open on Google Colab.*
2. *Do not delete or change variable names in the code cells below. You may add to each cell as many lines of code as you need, just make sure to assign your solution to the predefined variable(s) in the corresponding cell. Failing to do so will make the tests fail.*
3. *To save your work, click `File -> Save a copy on GitHub` and __make sure to manually select the correct repository from the dropdown list__.*
4. *If you mess up with this file and need to start from scratch, you can always find the original one [here](https://github.com/hse-mlwp-2022/assignment3-template/blob/main/pandas_exercise.ipynb). Just open it in Google Colab (see note 1) and save to your repository (see note 3). Remember to backup your code elsewhere, since this action will overwrite your previous work.* 
5. *Exercises 1-4 are mandatory. Your work __will not be graded__ if you fail any one of them. Exercises 5-8 are optional, you can skip them if you want*

## About the Dataset

We will be using 2019 flight statistics from the United States Department of Transportation’s Bureau of Transportation Statistics (available [here](https://www.transtats.bts.gov/DL_SelectFields.asp?gnoyr_VQ=FMF&QO_fu146_anzr=Nv4%20Pn44vr45) and in your repository as `data/T100_MARKET_ALL_CARRIER.zip`). You can load the dataset in pandas using this link: `https://github.com/hse-mlwp-2022/assignment3-template/raw/main/data/T100_MARKET_ALL_CARRIER.zip`.

Each row contains information about a specific route for a given carrier in a given month (e.g., JFK &rarr; LAX on Delta Airlines in January). There are 321,409 rows and 41 columns. Note that you don't need to unzip the file to read it in with `pd.read_csv()`.

#### Exercises

##### 1. Read in the data and convert the column names to lowercase to make them easier to work with.

In [ ]:
import pandas as pd
c = pd.read_csv('https://github.com/hse-mlwp-2022/assignment3-template/raw/main/data/T100_MARKET_ALL_CARRIER.zip')
c.rename(columns={col:col.lower() for col in c}, inplace = True)

##### 2. What columns are in the data? (0.5 point)

In [ ]:
columns = list(c.columns) 

print(columns)

##### 3. How many distinct carrier names are in the dataset? (0.5 point)

In [ ]:
carrier_names = c["carrier_name"].unique() 

print(carrier_names)

##### 4. Calculate the totals of the `freight`, `mail`, and `passengers` columns for flights from the United Kingdom to the United States. (1 point)

In [ ]:
freight_total = c[(c['origin_country'] == 'GB') & (c['dest_country'] == 'US')]['freight'].sum() 
mail_total = c[(c['origin_country'] == 'GB') & (c['dest_country'] == 'US')]['mail'].sum() 
passengers_total = c[(c['origin_country'] == 'GB') & (c['dest_country'] == 'US')]['passengers'].sum() 

print(f"freight total: {freight_total}")
print(f"mail total: {mail_total}")
print(f"passengers total: {passengers_total}")

##### 5. Which 10 carriers flew the most passengers out of the United States to another country? (1.5 points)
The result should be a Python iterable, e.g. a list or a corresponding pandas object

In [ ]:
import numpy as np
top_10_by_passengers = c[(c['origin_country'] == 'US') & (c['dest_country'] != 'US')].groupby('carrier_name').agg({'passengers':np.sum}).sort_values('passengers').reset_index()[-10:] 
print(f"List of top 10 carriers with max number of passengers flown out of US: {top_10_by_passengers}")

##### 6. Between which two cities were the most passengers flown? Make sure to account for both directions. (1.5 points)

In [ ]:
needed_entry = c.groupby(['origin_city_name', 'dest_city_name']).agg({'passengers':np.sum}).sort_values('passengers').reset_index().values[-1]
top_route_origin_city = needed_entry[0] 
top_route_dest_city = needed_entry[1] 
top_route_passengers_count = needed_entry[2] 

print(f"top route is '{top_route_origin_city} - {top_route_dest_city}' with traffic of {top_route_passengers_count} passengers")

##### 7. Find the top 3 carriers for the pair of cities found in #6 and calculate the percentage of passengers each accounted for. (2 points)
The result should be a pandas dataframe object with two columns: 
1. carrier name (string)
2. percentage of passengers (float in the range of 0-100)

In [ ]:
total_passengers = c[(c['origin_city_name'] == top_route_origin_city) & (c['dest_city_name'] == top_route_dest_city)]['passengers'].sum()
top_3_carriers_df = c[(c['origin_city_name'] == top_route_origin_city) & (c['dest_city_name'] == top_route_dest_city)].groupby('carrier_name').agg({'passengers' : (lambda x: np.sum(x) / total_passengers * 100)}).sort_values('passengers').reset_index()[-3:].rename(columns = {'carrier_name': 'carrier name', 'passengers' : 'percentage of passengers'}) # Place your code here instead of '...'

top_3_carriers_df

##### 8. Find the percentage of international travel per country using total passengers on class F flights. (3 points)

In [1]:
c1 = c[c['class'] == 'F'].groupby(['origin_country', 'dest_country']).agg({'passengers': np.sum}).reset_index()
c_int = c1[c1['origin_country'] != c1['dest_country']].groupby('origin_country').agg({'passengers': np.sum}).reset_index()
c_all = c1.groupby('origin_country').agg({'passengers': np.sum}).reset_index()
international_travel_per_country = pd.DataFrame([[x, c_int[c_int['origin_country'] == x]['passengers'].sum() / c_all[c_all['origin_country'] == x]['passengers'].sum() * 100] for x in c1['origin_country'].unique()], columns = ['origin country', 'percentage of international travel']) # Place your code here instead of '...'

international_travel_per_country

NameError: ignored